# 📊 abTEM 4D-STEM Simulation (FlexibleAnnularDetector)

This notebook simulates a 4D-STEM dataset using a DFT-relaxed 4H-SiC structure. It uses abTEM's `FlexibleAnnularDetector` to simulate the full scattering distribution and then integrates different angular ranges to obtain ABF, MAADF, and HAADF images.

---

## ✅ 1. Environment Setup (Google Colab)
Install exact package versions for compatibility:


In [ ]:
# Clean install for reproducibility
!pip uninstall -y numpy cupy abtem zarr -q
!pip install numpy==1.26.4
!pip install cupy-cuda117
!pip install abtem
!pip install zarr==2.14.2


## 📂 2. Upload Relaxed Structure (`relaxed_positions_only.cif`)

In [ ]:
from google.colab import files
files.upload()  # Upload your GPAW-relaxed CIF file


## 🧱 3. Build Supercell and Align Zone Axis

### 🧭 How to Choose and Align the Zone Axis

abTEM assumes the **beam propagates along the z-axis** of the simulation cell. This means:

- Your atomic structure must be aligned such that the **desired zone axis lies along the z-axis**.
- Use ASE's `surface()` function to extract a slab with the right orientation.
- Then rotate the structure so the beam aligns with the desired direction.

#### 🧱 Recommended Steps

```python
from ase.build import surface

atoms = read("relaxed_positions_only.cif")
atoms = surface(atoms, indices=(1, 0, 0), layers=4, periodic=True)  # zone axis [100]
atoms.rotate('x', 'z')  # aligns [100] along z-axis for abTEM
```

Then expand the slab if needed:
```python
atoms = atoms.repeat((2, 2, 1))  # xy expansion, z = beam direction
```

Finally, **center the slab in Y** so it doesn't clip the scan window:
```python
atoms.translate((0, -atoms.get_center_of_mass()[1] + atoms.cell[1, 1] / 2, 0))
```

---

💡 Choosing the correct zone axis (e.g., [100], [110]) depends on the crystal orientation you want to image. Use a structure viewer like VESTA or ASE GUI to determine the Miller indices that produce an interpretable zone axis.


In [ ]:
from ase.io import read
from ase.build import surface

atoms = read("relaxed_positions_only.cif")
atoms = surface(atoms, indices=(1, 0, 0), layers=4, periodic=True)  # create slab
atoms.rotate('x', 'z')  # beam direction along [100]
atoms = atoms.repeat((2, 2, 1))
atoms.wrap()
atoms.translate((0, -atoms.get_center_of_mass()[1] + atoms.cell[1, 1] / 2, 0))  # center in Y

print("Cell dimensions [Å]:", atoms.cell.lengths())


## 🔋 4. Create Potential Object

In [ ]:
import abtem
potential = abtem.Potential(atoms, sampling=0.025)
print("Potential grid shape:", potential.shape)


## ⚡ 5. Set Up GPU / CPU for FFT (CuPy or FFTW)

In [ ]:
import cupy
_ = cupy.zeros(1)
try:
    abtem.config.set({
        "device": "gpu",
        "fft": "cupy"
    })
    print("✅ Using GPU with CuPy.")
except Exception as e:
    abtem.config.set({
        "device": "cpu",
        "fft": "fftw"
    })
    print("⚠️ CuPy unavailable. Falling back to CPU. Reason:", e)


## 🔬 6. Define Probe

In [ ]:
probe = abtem.Probe(
    energy=300e3,              # 300 kV accelerating voltage
    semiangle_cutoff=20,       # mrad
    Cs=1.3e-3 * 1e10,          # 1.3 mm spherical aberration
    defocus=30                 # Defocus in Å (positive for overfocus)
)
probe.grid.match(potential)
print("Probe FWHM [Å]:", probe.profiles().width().compute())


### 🔬 abTEM `Probe` Parameter Guide

The probe in STEM defines how the electron beam is formed and how it interacts with the sample. Here's a breakdown of the parameters:

- `energy=300e3`: The accelerating voltage of the microscope in electron volts (eV).  
  Typical range: 60–300 keV depending on the system. 300 keV is standard for atomic-resolution STEM.

- `semiangle_cutoff=20`: The **convergence semi-angle** of the electron beam in milliradians (mrad).  
  It sets the **maximum scattering angle** simulated. Detectors should not exceed this value.

- `Cs=1.3e-3 * 1e10`: Spherical aberration coefficient (Cs), in Angstroms.  
  For example, 1.3 mm = 1.3e-3 m = `1.3e-3 * 1e10` Å. Used for probe shape modeling.

- `defocus=30`: The defocus in Angstroms. Positive = overfocus, negative = underfocus.  
  You may use `"scherzer"` instead of a number to auto-select the optimal value.

**Choosing semiangle cutoff:**  
- Ensure `semiangle_cutoff` is **greater than** the outer angle of any detector.
- Typical ranges:
  - ABF: 5–10 mrad
  - MAADF: 10–30 mrad
  - HAADF: 30–70+ mrad


## 🗺️ 7. Set Scan Grid

### 🗺️ `GridScan` Parameter Guide

`GridScan` defines the electron beam scan grid over your atomic model.

- `start=(0, 0), end=(1, 1)`: Defines scan bounds **as a fraction of the cell dimensions**.
    - `(0, 0)` is bottom-left, `(1, 1)` is top-right of the projected unit cell.
    - You can scan a subsection, e.g., `(0.2, 0.2)` to `(0.8, 0.8)`.

- `sampling=...`: Pixel spacing in Å (or fraction of Nyquist). Smaller = higher resolution.
    - `probe.aperture.nyquist_sampling` gives the ideal sampling to capture probe details.
    - You can reduce this (e.g., divide by 2) for higher resolution scans.

- `fractional=True`: Indicates that `start` and `end` are given as **fractions** of the potential size.
    - If `False`, they would be absolute coordinates in Å.

- `potential=...`: The potential object generated from your atomic structure.

**Scan resolution tip:**  
To reduce simulation time, use `sampling = nyquist / 2` or `nyquist / 3`. This balances detail and speed.


In [ ]:
from abtem import GridScan
scan = GridScan(
    start=(0, 0), end=(1, 1),
    sampling=probe.aperture.nyquist_sampling / 2,
    fractional=True,
    potential=potential
)
print("Scan shape:", scan.shape)


## 🎯 8. Run Simulation with Flexible Detector

In [ ]:
from abtem.detectors import FlexibleAnnularDetector
flex = FlexibleAnnularDetector()

result = probe.scan(potential, scan=scan, detectors=flex)
result.compute()
result.save("flex_scan.zarr")
print("Saved full scan to flex_scan.zarr")


## 🎞️ 9. Integrate Radial Ranges for STEM Images

### 🎯 Detector Angle Selection and Integration Ranges

In 4D-STEM with `FlexibleAnnularDetector`, we capture the **full angular scattering distribution**, and later extract signals by integrating over angle ranges:

```python
abf = result.integrate_radial(0, 10)     # ABF
maadf = result.integrate_radial(10, 20)  # MAADF
haadf = result.integrate_radial(20, 30)  # HAADF
```

⚠️ **Important:** These angular values must not exceed the `probe.semiangle_cutoff` value. If `semiangle_cutoff=20`, then a range like `(20, 30)` will fail or give no signal.

#### 🧠 Recommended Ranges:
| Signal Type | Suggested Angle Range (mrad) |
|-------------|------------------------------|
| ABF         |  0 – 10                      |
| MAADF       | 10 – 20                      |
| HAADF       | 20 – 30 (only if cutoff > 30)|

You can check this before integration:
```python
print("Max angle simulated:", probe.semiangle_cutoff, "mrad")
```

Always ensure your detector ranges fall within `0` to `semiangle_cutoff`.


In [ ]:
from abtem import stack

abf = result.integrate_radial(0, 10)     # ABF
maadf = result.integrate_radial(10, 20)  # MAADF
haadf = result.integrate_radial(20, 30)  # HAADF

measurement_stack = stack([abf, maadf, haadf])
fig = measurement_stack.show(
    explode=True, figsize=(14, 5), cbar=True, cmap="gray", return_figure=True
)
fig.savefig("stacked_measurements.png", dpi=300)
print("Saved image as stacked_measurements.png")
